## Rakenna logistinen regressiomalli - Oppitunti 4

![Logistinen vs. lineaarinen regressio - infografiikka](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[Esiluennon kysely](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Johdanto

Tässä viimeisessä oppitunnissa regressiosta, joka on yksi perus *klassinen* ML-tekniikka, tarkastelemme logistista regressiota. Tätä tekniikkaa käytetään mallintamaan ja ennustamaan binäärisiä kategorioita. Onko tämä karkki suklaata vai ei? Onko tämä tauti tarttuva vai ei? Valitseeko asiakas tämän tuotteen vai ei?

Tässä oppitunnissa opit:

-   Tekniikoita logistiseen regressioon

✅ Syvennä ymmärrystäsi tämän tyyppisen regression käytöstä tässä [Learn-moduulissa](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Esitiedot

Työskenneltyämme kurpitsadataa kanssa olemme nyt riittävän tuttuja sen kanssa huomataksemme, että siinä on yksi binäärinen kategoria, jonka kanssa voimme työskennellä: `Color`.

Rakennetaan logistinen regressiomalli ennustamaan, annettujen muuttujien perusteella, *minkä värinen tietty kurpitsa todennäköisesti on* (oranssi 🎃 vai valkoinen 👻).

> Miksi puhumme binäärisestä luokittelusta regressio-oppituntien yhteydessä? Ainoastaan kielellisen mukavuuden vuoksi, sillä logistinen regressio on [todellisuudessa luokittelumenetelmä](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), vaikkakin lineaarinen. Opit muita tapoja luokitella dataa seuraavassa oppituntiryhmässä.

Tätä oppituntia varten tarvitsemme seuraavat paketit:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) on [R-pakettien kokoelma](https://www.tidyverse.org/packages), joka tekee data-analytiikasta nopeampaa, helpompaa ja hauskempaa!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) -kehys on [pakettikokoelma](https://www.tidymodels.org/packages/) mallinnukseen ja koneoppimiseen.

-   `janitor`: [janitor-paketti](https://github.com/sfirke/janitor) tarjoaa yksinkertaisia työkaluja likaisen datan tarkasteluun ja puhdistamiseen.

-   `ggbeeswarm`: [ggbeeswarm-paketti](https://github.com/eclarke/ggbeeswarm) tarjoaa menetelmiä beeswarm-tyylisten kaavioiden luomiseen ggplot2:lla.

Voit asentaa ne seuraavasti:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Vaihtoehtoisesti alla oleva skripti tarkistaa, onko sinulla tarvittavat paketit tämän moduulin suorittamiseen, ja asentaa ne puolestasi, jos ne puuttuvat.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Määrittele kysymys**

Tässä tapauksessa ilmaisemme sen binäärisesti: 'Valkoinen' tai 'Ei valkoinen'. Datasetissämme on myös 'raidallinen' kategoria, mutta sen esiintymiä on vähän, joten emme käytä sitä. Se katoaa joka tapauksessa, kun poistamme datasetistä null-arvot.

> 🎃 Hauska fakta: kutsumme joskus valkoisia kurpitsoja 'haamukurpitsoiksi'. Niitä ei ole kovin helppo kaivertaa, joten ne eivät ole yhtä suosittuja kuin oranssit, mutta ne näyttävät siisteiltä! Voisimme siis muotoilla kysymyksemme myös näin: 'Haamu' vai 'Ei haamu'. 👻

## **Tietoa logistisesta regressiosta**

Logistinen regressio eroaa lineaarisesta regressiosta, josta opit aiemmin, muutamalla tärkeällä tavalla.

#### **Binääriluokittelu**

Logistinen regressio ei tarjoa samoja ominaisuuksia kuin lineaarinen regressio. Edellinen antaa ennusteen `binäärisestä kategoriasta` ("oranssi tai ei oranssi"), kun taas jälkimmäinen pystyy ennustamaan `jatkuvia arvoja`, esimerkiksi kurpitsan alkuperän ja sadonkorjuuajan perusteella, *kuinka paljon sen hinta nousee*.

![Infografiikka: Dasani Madipalli](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### Muut luokittelut

Logistisessa regressiossa on myös muita tyyppejä, kuten multinomiaalinen ja ordinaalinen:

- **Multinomiaalinen**, jossa on useampi kuin yksi kategoria - "Oranssi, Valkoinen ja Raidallinen".

- **Ordinaalinen**, jossa on järjestettyjä kategorioita. Tämä on hyödyllistä, jos haluamme järjestää tulokset loogisesti, kuten kurpitsat, jotka on järjestetty rajallisen määrän kokoluokkien mukaan (mini, sm, med, lg, xl, xxl).

![Multinomiaalinen vs ordinaalinen regressio](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **Muuttujien EI tarvitse korreloida**

Muistatko, kuinka lineaarinen regressio toimi paremmin, kun muuttujat korreloivat enemmän? Logistinen regressio on päinvastainen - muuttujien ei tarvitse olla linjassa. Tämä sopii tälle datalle, jossa korrelaatiot ovat melko heikkoja.

#### **Tarvitset paljon puhdasta dataa**

Logistinen regressio antaa tarkempia tuloksia, jos käytät enemmän dataa; pieni datasetimme ei ole optimaalinen tähän tehtävään, joten pidä tämä mielessä.

✅ Mieti, millaiset datatyypit sopisivat hyvin logistiseen regressioon.

## Harjoitus - siivoa data

Ensiksi siivoa dataa hieman, poista null-arvot ja valitse vain osa sarakkeista:

1. Lisää seuraava koodi:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Voit aina vilkaista uutta dataframeasi käyttämällä [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html)-funktiota seuraavasti:


In [ ]:
pumpkins_select %>% 
  glimpse()


Varmistetaan, että kyseessä on todellakin binääriluokitusongelma:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Visualisointi - kategorinen kaavio
Olet nyt ladannut kurpitsadatan uudelleen ja siivonnut sen niin, että datasettiin on säilytetty muutamia muuttujia, mukaan lukien Väri. Visualisoidaan dataframe muistikirjassa käyttämällä ggplot-kirjastoa.

Ggplot-kirjasto tarjoaa käteviä tapoja visualisoida dataasi. Esimerkiksi voit vertailla datan jakaumia jokaisen Lajikkeen ja Värin osalta kategorisessa kaaviossa.

1. Luo tällainen kaavio käyttämällä geombar-funktiota, hyödyntäen kurpitsadataamme, ja määritä värikartoitus jokaiselle kurpitsakategorialle (oranssi tai valkoinen):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

Tarkastelemalla tietoja voit nähdä, miten Väri-data liittyy Lajikkeeseen.

✅ Kun tarkastelet tätä kategorista kaaviota, millaisia mielenkiintoisia tutkimusideoita voit kuvitella?


### Datan esikäsittely: piirteiden koodaus

Kurpitsadatasetissamme kaikki sarakkeet sisältävät merkkijonoarvoja. Kategorisen datan käsittely on ihmisille intuitiivista, mutta koneille ei. Koneoppimisalgoritmit toimivat hyvin numeroiden kanssa. Siksi koodaus on erittäin tärkeä vaihe datan esikäsittelyssä, sillä sen avulla voimme muuntaa kategorisen datan numeeriseksi menettämättä tietoa. Hyvä koodaus auttaa rakentamaan hyvän mallin.

Piirteiden koodaukseen on kaksi päätyyppiä:

1. Ordinaalikoodaus: Tämä sopii hyvin ordinaalisille muuttujille, jotka ovat kategorisia muuttujia, joiden data noudattaa loogista järjestystä, kuten `item_size`-sarake datasetissämme. Se luo kartoituksen, jossa jokainen kategoria esitetään numerolla, joka vastaa kategorian järjestystä sarakkeessa.

2. Kategorinen koodaus: Tämä sopii hyvin nominaalisille muuttujille, jotka ovat kategorisia muuttujia, joiden data ei noudata loogista järjestystä, kuten kaikki muut piirteet datasetissamme paitsi `item_size`. Tämä on niin sanottu one-hot-koodaus, mikä tarkoittaa, että jokainen kategoria esitetään binäärisarakkeena: koodattu muuttuja on arvoltaan 1, jos kurpitsa kuuluu kyseiseen Varietyyn, ja 0 muuten.

Tidymodels tarjoaa vielä yhden kätevän paketin: [recipes](https://recipes.tidymodels.org/) - paketti datan esikäsittelyyn. Määrittelemme `recipe`-objektin, joka määrittää, että kaikki ennustajasarakkeet tulisi koodata kokonaisluvuiksi, `prep`-toiminnolla arvioimme tarvittavat määrät ja tilastot operaatioita varten, ja lopuksi `bake`-toiminnolla sovellamme laskelmat uuteen dataan.

> Yleensä recipes-pakettia käytetään esiprosessorina mallinnuksessa, jossa se määrittää, mitä vaiheita dataan tulisi soveltaa, jotta se olisi valmis mallinnukseen. Tässä tapauksessa on **erittäin suositeltavaa**, että käytät `workflow()`-toimintoa sen sijaan, että arvioisit reseptin manuaalisesti prep- ja bake-toiminnoilla. Näemme tämän kaiken hetken kuluttua.
>
> Tällä kertaa käytämme kuitenkin recipes + prep + bake -yhdistelmää määrittääksemme, mitä vaiheita dataan tulisi soveltaa, jotta se olisi valmis data-analyysiin, ja sitten poimimme esikäsitellyn datan sovelletuilla vaiheilla.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Mitkä ovat edut käyttää järjestysenkooderia Item Size -sarakkeelle?

### Analysoi muuttujien välisiä suhteita

Kun olemme esikäsitelleet datamme, voimme analysoida ominaisuuksien ja tunnisteen välisiä suhteita saadaksemme käsityksen siitä, kuinka hyvin malli pystyy ennustamaan tunnisteen ominaisuuksien perusteella. Paras tapa suorittaa tällainen analyysi on datan visualisointi. 
Käytämme jälleen ggplotin geom_boxplot_-funktiota, jotta voimme visualisoida Item Size-, Variety- ja Color-sarakkeiden välisiä suhteita kategorisessa kaaviossa. Jotta voimme paremmin visualisoida dataa, käytämme enkoodattua Item Size -sarakeetta ja enkoodaamatonta Variety-saraketta.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Käytä parviplottia

Koska Väri on binäärinen kategoria (Valkoinen tai Ei), sen visualisointiin tarvitaan '[erikoistunut lähestymistapa](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)'.

Kokeile `parviplottia` näyttääksesi värin jakautumisen suhteessa item_size-arvoon.

Käytämme [ggbeeswarm-pakettia](https://github.com/eclarke/ggbeeswarm), joka tarjoaa menetelmiä parvityylisten plottien luomiseen ggplot2:n avulla. Parviplotit ovat tapa esittää pisteitä, jotka normaalisti menisivät päällekkäin, niin että ne asettuvat vierekkäin.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Nyt kun meillä on käsitys värin binääristen kategorioiden ja suuremman kokoryhmän välisestä suhteesta, tutkitaan logistista regressiota määrittääksemme todennäköisen värin tietylle kurpitsalle.

## Rakenna mallisi

Valitse muuttujat, joita haluat käyttää luokittelumallissasi, ja jaa data koulutus- ja testijoukkoihin. [rsample](https://rsample.tidymodels.org/), Tidymodels-paketti, tarjoaa infrastruktuurin tehokkaaseen datan jakamiseen ja uudelleennäytteistämiseen:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 Olemme nyt valmiita kouluttamaan mallin sovittamalla harjoitusominaisuudet harjoitusmerkkiin (väri).

Aloitamme luomalla reseptin, joka määrittelee esikäsittelyvaiheet, jotka datalle tulee suorittaa, jotta se olisi valmis mallinnusta varten, esimerkiksi: kategoristen muuttujien koodaaminen kokonaisluvuiksi. Aivan kuten `baked_pumpkins`, luomme `pumpkins_recipe`, mutta emme käytä `prep`- ja `bake`-toimintoja, koska ne sisällytetään työnkulkuun, jonka näet muutaman askeleen päästä.

Tidymodelsissa on useita tapoja määritellä logistinen regressiomalli. Katso `?logistic_reg()`. Tällä kertaa määrittelemme logistisen regressiomallin oletusarvoisen `stats::glm()`-moottorin avulla.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Nyt kun meillä on resepti ja mallin määrittely, meidän täytyy löytää tapa yhdistää ne yhdeksi objektiksi, joka ensin esikäsittelee datan (valmistelu + leivonta kulissien takana), sovittaa mallin esikäsiteltyyn dataan ja mahdollistaa myös mahdolliset jälkikäsittelytoimet.

Tidymodels-paketissa tämä kätevä objekti kutsutaan [`workflow`](https://workflows.tidymodels.org/), ja se pitää kätevästi sisällään mallinnuskomponenttisi.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Kun työnkulku on *määritelty*, mallin voi `kouluttaa` käyttämällä [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html)-funktiota. Työnkulku arvioi reseptin ja esikäsittelee datan ennen koulutusta, joten meidän ei tarvitse tehdä sitä manuaalisesti käyttämällä prep- ja bake-funktioita.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


Mallin tulostus näyttää koulutuksen aikana opitut kertoimet.

Nyt kun olemme kouluttaneet mallin käyttämällä harjoitusaineistoa, voimme tehdä ennusteita testiaineistosta käyttäen [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Aloitetaan käyttämällä mallia ennustamaan testijoukon luokkia ja kunkin luokan todennäköisyyksiä. Kun todennäköisyys on yli 0.5, ennustettu luokka on `WHITE`, muuten `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Todella hienoa! Tämä antaa lisää näkemyksiä siitä, miten logistinen regressio toimii.

### Parempi ymmärrys sekaannusmatriisin avulla

Jokaisen ennusteen vertaaminen vastaavaan "todelliseen" arvoon ei ole kovin tehokas tapa arvioida, kuinka hyvin malli ennustaa. Onneksi Tidymodels tarjoaa muutamia lisätyökaluja: [`yardstick`](https://yardstick.tidymodels.org/) - paketti, jota käytetään mallien tehokkuuden mittaamiseen suorituskykymittareiden avulla.

Yksi suorituskykymittari, joka liittyy luokittelutehtäviin, on [`sekaannusmatriisi`](https://wikipedia.org/wiki/Confusion_matrix). Sekaannusmatriisi kuvaa, kuinka hyvin luokittelumalli toimii. Sekaannusmatriisi laskee, kuinka monta esimerkkiä kustakin luokasta malli luokitteli oikein. Meidän tapauksessamme se näyttää, kuinka monta oranssia kurpitsaa luokiteltiin oransseiksi ja kuinka monta valkoista kurpitsaa luokiteltiin valkoisiksi; sekaannusmatriisi näyttää myös, kuinka monta luokiteltiin **väärin** kategorioihin.

[**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html)-funktio yardstick-paketista laskee tämän havaittujen ja ennustettujen luokkien ristiintaulukoinnin.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Tulkitkaamme sekaannusmatriisia. Mallimme tehtävänä on luokitella kurpitsat kahteen binaariseen kategoriaan: kategoriaan `valkoinen` ja kategoriaan `ei-valkoinen`.

-   Jos mallisi ennustaa kurpitsan olevan valkoinen ja se kuuluu todellisuudessa kategoriaan 'valkoinen', kutsumme sitä `tosipositiviseksi`, joka näkyy vasemmassa yläkulmassa.

-   Jos mallisi ennustaa kurpitsan olevan ei-valkoinen ja se kuuluu todellisuudessa kategoriaan 'valkoinen', kutsumme sitä `vääräksi negatiiviseksi`, joka näkyy vasemmassa alakulmassa.

-   Jos mallisi ennustaa kurpitsan olevan valkoinen ja se kuuluu todellisuudessa kategoriaan 'ei-valkoinen', kutsumme sitä `vääräksi positiviseksi`, joka näkyy oikeassa yläkulmassa.

-   Jos mallisi ennustaa kurpitsan olevan ei-valkoinen ja se kuuluu todellisuudessa kategoriaan 'ei-valkoinen', kutsumme sitä `tosinegatiiviseksi`, joka näkyy oikeassa alakulmassa.

| Totuus |
|:-----:|


|               |        |       |
|---------------|--------|-------|
| **Ennustettu** | VALKOINEN | ORANSSI |
| VALKOINEN      | TP     | FP    |
| ORANSSI        | FN     | TN    |

Kuten ehkä arvasit, on suotavaa, että tosi-positiivisten ja tosi-negatiivisten määrä on suuri ja väärien positiivisten ja väärien negatiivisten määrä pieni, mikä tarkoittaa, että malli suoriutuu paremmin.

Sekaannusmatriisi on hyödyllinen, koska sen avulla voidaan laskea muita mittareita, jotka auttavat arvioimaan luokittelumallin suorituskykyä paremmin. Käydään läpi joitakin niistä:

🎓 Tarkkuus (Precision): `TP/(TP + FP)` määritellään ennustettujen positiivisten osuutena, jotka ovat oikeasti positiivisia. Kutsutaan myös nimellä [positiivinen ennustearvo](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 Herkkyys (Recall): `TP/(TP + FN)` määritellään positiivisten tulosten osuutena todellisuudessa positiivisista näytteistä. Tunnetaan myös nimellä `sensitiivisyys`.

🎓 Spesifisyys: `TN/(TN + FP)` määritellään negatiivisten tulosten osuutena todellisuudessa negatiivisista näytteistä.

🎓 Tarkkuus (Accuracy): `TP + TN/(TP + TN + FP + FN)` Oikein ennustettujen näytteiden prosenttiosuus kaikista näytteistä.

🎓 F-mitta: Tarkkuuden ja herkkyyden painotettu keskiarvo, jossa paras arvo on 1 ja huonoin 0.

Lasketaan nämä mittarit!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Visualisoi tämän mallin ROC-käyrä

Tehdään vielä yksi visualisointi, jotta voimme tarkastella niin kutsuttua [`ROC-käyrää`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC-käyrät ovat usein käytössä, kun halutaan tarkastella luokittelijan tuloksia todellisten ja väärien positiivisten osalta. ROC-käyrissä Y-akselilla on yleensä `True Positive Rate`/herkkyys ja X-akselilla `False Positive Rate`/1-spesifisyys. Käyrän jyrkkyys ja etäisyys keskilinjasta ovat tärkeitä: halutaan käyrä, joka nopeasti nousee ylös ja ylittää linjan. Meidän tapauksessamme alussa on vääriä positiivisia, mutta sen jälkeen käyrä nousee ja ylittää linjan asianmukaisesti.

Lopuksi käytetään `yardstick::roc_auc()`-funktiota laskemaan todellinen käyrän alle jäävä pinta-ala (Area Under the Curve). Yksi tapa tulkita AUC-arvoa on pitää sitä todennäköisyytenä, että malli sijoittaa satunnaisen positiivisen esimerkin korkeammalle kuin satunnaisen negatiivisen esimerkin.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Tulos on noin `0.975`. Koska AUC vaihtelee välillä 0–1, haluat korkean pistemäärän, sillä malli, joka on 100 % oikeassa ennusteissaan, saa AUC-arvon 1; tässä tapauksessa malli on *melko hyvä*.

Tulevissa luokittelua käsittelevissä oppitunneissa opit, kuinka parantaa mallisi tuloksia (kuten käsitellä epätasapainoista dataa tässä tapauksessa).

## 🚀Haaste

Logistiseen regressioon liittyy paljon enemmän kuin tässä käsiteltiin! Paras tapa oppia on kokeilla. Etsi datasetti, joka sopii tämän tyyppiseen analyysiin, ja rakenna malli sen avulla. Mitä opit? vinkki: kokeile [Kagglea](https://www.kaggle.com/search?q=logistic+regression+datasets) löytääksesi mielenkiintoisia datasettejä.

## Kertaus & Itseopiskelu

Lue muutama ensimmäinen sivu [tästä Stanfordin artikkelista](https://web.stanford.edu/~jurafsky/slp3/5.pdf), jossa käsitellään logistisen regression käytännön sovelluksia. Mieti tehtäviä, jotka sopivat paremmin joko yhdelle tai toiselle regressiotyypille, joita olemme tähän mennessä opiskelleet. Mikä toimisi parhaiten?



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskääntämistä. Emme ole vastuussa tämän käännöksen käytöstä aiheutuvista väärinkäsityksistä tai virhetulkinnoista.
